In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision.datasets import ImageFolder
import flwr as fl
from torchsummary import summary
DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)
from sklearn.metrics import classification_report

Training on cuda using PyTorch 2.0.0+cu118 and Flower 1.4.0


In [2]:
NUM_CLIENTS = 5
hw = 128

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(size = (hw,hw), antialias=True),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
    ]
)

In [4]:
def findConv2dOutShape(hin,win,conv,pool=2):
    # get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation

    hout=np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    wout=np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

    if pool:
        hout/=pool
        wout/=pool
    return int(hout),int(wout)

class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        Cin,Hin,Win= (3, hw, hw)
        init_f= 32
        num_fc1= 512
        num_classes= 2
        self.dropout_rate= 0.2
        
        # Convolution Layers
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
        h,w=findConv2dOutShape(Hin,Win,self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv3)
        self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv4)
        self.conv5 = nn.Conv2d(8*init_f, 16*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv5)
        
        self.num_flatten=h*w*16*init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        # Convolution & Pool Layers
        X = F.relu(self.conv1(X)); 
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv5(X))
        X = F.max_pool2d(X, 2, 2) 
        
        X = X.view(-1, self.num_flatten)
        
        X = F.relu(self.fc1(X))
        X=F.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return F.log_softmax(X, dim=1)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int, cid=None):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    
    with open("client-log-"+str(cid)+".txt", "a") as file:
        file.write("=======ROUND START======= \n")

    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        
        with open("client-log-"+str(cid)+".txt", "a") as file:
            file.write(f"{epoch_loss},{epoch_acc} \n")
            
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")

            


def test(net, testloader, cid):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    y_test = []
    y_pred = []
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            y_test = y_test + labels.cpu().numpy().tolist()
            y_pred = y_pred + predicted.cpu().numpy().tolist()
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    
    with open("client-report-"+str(cid)+".txt", "a") as file:
        file.write(classification_report(y_test, y_pred, zero_division=0) + "\n")
    
    
    accuracy = correct / total
    return loss, accuracy

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=10, cid=self.cid)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader, cid=self.cid)
        with open("client-result-"+str(self.cid)+".txt", "a") as file:
            file.write(f"{loss}, {accuracy} \n")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)

    train_dataset = ImageFolder(root='D:/feda/partitions/client-'+str(cid)+'/split/train',
        transform=transform)
    val_dataset = ImageFolder(root='D:/feda/partitions/client-'+str(cid)+'/split/val',
        transform=transform)
    
    
    trainDataLoader = DataLoader(train_dataset, 
            batch_size=32, shuffle=True)
    valDataLoader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    return FlowerClient(cid, net, trainDataLoader, valDataLoader)

In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1,
    fraction_evaluate=1,
    min_fit_clients=5,
    min_evaluate_clients=5,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=50),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-28 16:44:09,611 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=50, round_timeout=None)
2023-04-28 16:44:12,270	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-04-28 16:44:15,179 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 2091668275.0, 'memory': 4183336551.0, 'CPU': 8.0, 'GPU': 1.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-04-28 16:44:15,180 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-28 16:44:15,180 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-04-28 16:44:15,181 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-28 16:44:15,181 | server.py:101 | FL starting
DEBUG flwr 2023-04-28 16:44:15,182 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


(launch_and_fit pid=72564) [Client 3] fit, config: {}
(launch_and_fit pid=72564) Epoch 1: train loss 0.021969977766275406, accuracy 0.4964850615114236
(launch_and_fit pid=72564) Epoch 2: train loss 0.0218733549118042, accuracy 0.5307557117750439
(launch_and_fit pid=72564) Epoch 3: train loss 0.02203921042382717, accuracy 0.5272407732864675
(launch_and_fit pid=72564) Epoch 4: train loss 0.021908553317189217, accuracy 0.5342706502636204
(launch_and_fit pid=72564) Epoch 5: train loss 0.02198130078613758, accuracy 0.4991212653778559
(launch_and_fit pid=72564) Epoch 6: train loss 0.021932510659098625, accuracy 0.5026362038664324
(launch_and_fit pid=72564) Epoch 7: train loss 0.02194458618760109, accuracy 0.5061511423550088
(launch_and_fit pid=72564) Epoch 8: train loss 0.021906478330492973, accuracy 0.5052724077328646
(launch_and_fit pid=72564) Epoch 9: train loss 0.0219383854418993, accuracy 0.5281195079086116
(launch_and_fit pid=72564) Epoch 10: train loss 0.021922433748841286, accuracy 0

DEBUG flwr 2023-04-28 16:46:45,096 | server.py:232 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-04-28 16:46:45,172 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-28 16:46:45,177 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


(launch_and_fit pid=26896) Epoch 10: train loss 0.021885059773921967, accuracy 0.4982425307557118
(launch_and_evaluate pid=73244) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=73188) [Client 1] evaluate, config: {} [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(launch_and_evaluate pid=65596) [Client 3] evaluate, config: {} [repeated 2x across cluster]


DEBUG flwr 2023-04-28 16:47:08,883 | server.py:182 | evaluate_round 1 received 5 results and 0 failures
WARNING flwr 2023-04-28 16:47:08,884 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-04-28 16:47:08,885 | server.py:218 | fit_round 2: strategy sampled 5 clients (out of 5)


...

DEBUG flwr 2023-04-28 18:41:48,010 | server.py:232 | fit_round 43 received 5 results and 0 failures
DEBUG flwr 2023-04-28 18:41:48,098 | server.py:168 | evaluate_round 43: strategy sampled 5 clients (out of 5)


(launch_and_fit pid=63376) Epoch 10: train loss 0.00506597850471735, accuracy 0.9367311072056239
(autoscaler +1h57m43s) Warning: The following resource request cannot be scheduled right now: {'GPU': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(launch_and_evaluate pid=73880) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=72272) [Client 0] evaluate, config: {} [repeated 2x across cluster]
(launch_and_evaluate pid=63864) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=71612) [Client 4] evaluate, config: {}


DEBUG flwr 2023-04-28 18:42:12,805 | server.py:182 | evaluate_round 43 received 5 results and 0 failures
DEBUG flwr 2023-04-28 18:42:12,805 | server.py:218 | fit_round 44: strategy sampled 5 clients (out of 5)


(launch_and_fit pid=65516) [Client 1] fit, config: {}
(launch_and_fit pid=65516) Epoch 1: train loss 0.003809594549238682, accuracy 0.960456942003515
(launch_and_fit pid=65516) Epoch 2: train loss 0.003119563451036811, accuracy 0.9710017574692443
(launch_and_fit pid=65516) Epoch 3: train loss 0.004491652362048626, accuracy 0.953427065026362
(launch_and_fit pid=65516) Epoch 4: train loss 0.003455798840150237, accuracy 0.9560632688927944
(launch_and_fit pid=65516) Epoch 5: train loss 0.0027906950563192368, accuracy 0.9727592267135325
(launch_and_fit pid=65516) Epoch 6: train loss 0.0026505128480494022, accuracy 0.9666080843585237
(launch_and_fit pid=65516) Epoch 7: train loss 0.002212086459621787, accuracy 0.9701230228471002
(launch_and_fit pid=65516) Epoch 8: train loss 0.0019538188353180885, accuracy 0.9806678383128296
(launch_and_fit pid=65516) Epoch 9: train loss 0.002991489600390196, accuracy 0.9578207381370826
(launch_and_fit pid=65516) Epoch 10: train loss 0.0023287120275199413, a

Exception in thread ray_listen_error_messages:
Traceback (most recent call last):
  File "C:\Users\muaz\anaconda3\envs\f2\lib\threading.py", line 980, in _bootstrap_inner
2023-04-28 18:43:21,013	ERROR import_thread.py:83 -- ImportThread: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:60806 {created_time:"2023-04-28T12:43:20.963048718+00:00", grpc_status:2, grpc_message:"Stream removed"}"
>
2023-04-28 18:43:21,022	ERROR worker.py:894 -- print_logs: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:60806 {grpc_message:"Stream removed", grpc_status:2, created_time:"2023-04-28T12:43:20.96364143+00:00"}"
>
    self.run()
  File "C:\Users\muaz\anaconda3\envs\f2\lib\threading.py", line 917, in run
